In [ ]:
!ls dataset.zip || {wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V_a2wmNp3o02X2j0PONTtCCPCOn-kiJl" -O dataset.zip && rm -rf /tmp/cookies.txt}

In [ ]:
!ls dataset/ || unzip dataset.zip

In [ ]:
import pandas as pd

In [ ]:
network_normal_data = pd.read_csv("dataset/Network datatset/csv/normal.csv")

In [ ]:
physical_normal_data = pd.read_csv("dataset/Physical dataset/phy_norm.csv", encoding='utf-16')

In [ ]:
network_normal_data

In [ ]:
physical_normal_data # Is going to need a lot of preprocessing ... use `file` on all physical files to verify their encoding... a function would be great I think

In [ ]:
# TODO:
# Make a function to clean the physical datasets
# Make datasets for train and test with samples (I suspect using everything would be too much for my poor cpu)
# Train some models: Isolation Forest, LOF, KNN, SVM, Naive Bayes
# Generate basic statistics and visualisations
# This would be enough for 26.10 and should not be long
# Why not train a MLP :eyes: ?